In [1]:
# Standard libraries
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from signal_lib import resample
from corr_lib import sprMat
from qt_wrapper import gui_fnames, gui_fpath

from matlab.h5data_lib import readTE_H5, getStatistics, parseTEfolders, getTitlesAndIndices
from matlab.matlab_yaro_lib import read_neuro_perf, read_paw, read_lick, read_whisk
from matlab.parse_folders import parseFoldersMulti
from matlab.preprocess_behaviour import resample_lick, resample_paw, resample_whisk

/opt/anaconda3/envs/py36qt5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Appended library directory /media/aleksejs/DataHDD/work/codes/theoretical-models/experimental-bridge/nest-mesoscopic-metric-test/codes/lib


In [2]:
%load_ext autoreload
%autoreload 2

from matlab.parse_folders import parseFoldersMulti

# Automatically find subfolders for data and behaviour

In [3]:
tmp_path = root_path_data if 'root_path_data' in locals() else "./"
root_path_data  = gui_fpath("Path to data collection", tmp_path)
root_path_paw   = gui_fpath("Path to Paw data",        root_path_data)
root_path_lick  = gui_fpath("Path to Lick data",       root_path_paw)
root_path_whisk = gui_fpath("Path to Whisk Data",      root_path_lick)

In [4]:
micePathArr, summary = parseFoldersMulti(root_path_data, root_path_paw, root_path_lick, root_path_whisk)
mice = list(set(micePathArr[:, 0]))

print("Number of day folders for each mouse")
pd.DataFrame(summary, index=mice)

Number of day folders for each mouse


,Data,Lick,Paw,Whisk
mtp_11,12,15,6,11
mtp_16,13,13,0,14
mtp_12,24,20,30,17
mtp_10,8,10,11,8
mtp_9,0,7,4,7
mtp_1,9,11,0,11
mtp_7,18,23,4,24
mtp_8,0,0,0,3
mtp_2,8,8,0,10
mtp_13,12,13,8,10


# Read available data and behaviour, resample

In [5]:
def findPathIdx(typeKey, mouseDayKey):
    rez = np.where((micePathArr[:, 1] == typeKey)&(micePathArr[:, 3] == mouseDayKey))[0]
    if len(rez)==0:
        return None
    elif len(rez)==1:
        return rez[0]
    else:
        raise ValueError("Expected 0 or 1 results, got", len(rez))
        
def findDataIdx(pathIdx):
    return np.where(miceDataIdxs == pathIdx)[0][0]

In [6]:
# Read Neuronal data
miceDataIdxs = np.where(micePathArr[:, 1] == 'data_path')[0]
dataNeuronal = []
dataTrials   = []
progBar = IntProgress(min=0, max=miceDataIdxs.shape[0], description='Read Neuro Data:')
display(progBar) # display the bar
for datapath in micePathArr[miceDataIdxs, 2]:
    data, behaviour = read_neuro_perf(datapath, verbose=False)
    dataNeuronal += [data]
    dataTrials   += [behaviour]
    progBar.value += 1
    
# Define resampling frequency
TARGET_RANGE = [0, 8] # Seconds goal
TARGET_FREQ = 20 # Hz
TARGET_NTIMES = int((TARGET_RANGE[1]-TARGET_RANGE[0])*TARGET_FREQ) + 1
TARGET_TIMES = np.linspace(TARGET_RANGE[0], TARGET_RANGE[1], TARGET_NTIMES)
print("Target trial within", TARGET_RANGE, "sec. Total target timesteps", TARGET_NTIMES)
    
# Read and resample paw
micePawIdxs = np.where(micePathArr[:, 1] == 'paw_path')[0]
dataPawResampled = []
progBar = IntProgress(min=0, max=micePawIdxs.shape[0], description='Read paw data:')
display(progBar) # display the bar
for pawpath in micePathArr[micePawIdxs, 2]:
    dataPaw = read_paw(pawpath, verbose=False)
    dataPawResampled += [resample_paw(dataPaw, TARGET_TIMES, TARGET_FREQ)]
    progBar.value += 1

# Read and resample lick
miceLickIdxs = np.where(micePathArr[:, 1] == 'lick_path')[0]
dataLickResampled = []
progBar = IntProgress(min=0, max=miceLickIdxs.shape[0], description='Read lick data:')
display(progBar) # display the bar
for lickpath, lickbase in micePathArr[miceLickIdxs, 2:4]:
    # Find behaviour associated with this lick
    pathIdx = findPathIdx('data_path', lickbase)
    if pathIdx is None:
        dataLickResampled += [None]
    else:
        dataIdx = np.where(miceDataIdxs == pathIdx)[0][0]
        neuro = dataNeuronal[dataIdx]
        behaviour = dataTrials[dataIdx]
        dataLick = read_lick(lickpath, verbose=False)
        dataLickResampled += [resample_lick(dataLick, neuro, behaviour, TARGET_TIMES, TARGET_FREQ)]
    progBar.value += 1


# Read and resample whisk
miceWhiskIdxs = np.where(micePathArr[:, 1] == 'whisk_path')[0]
progBar = IntProgress(min=0, max=miceWhiskIdxs.shape[0], description='Read whisk data:')
display(progBar) # display the bar
dataWhiskResampled = []
for whiskpath in micePathArr[miceWhiskIdxs, 2]:
    dataWhisk = read_whisk(whiskpath, verbose=False)
    dataWhiskResampled += [resample_whisk(dataWhisk, TARGET_TIMES)]
    progBar.value += 1

IntProgress(value=0, description='Read Neuro Data:', max=117)

Behaviour has more trials than data 438 437
Behaviour max index must be less than number of trials 437 437
Target trial within [0, 8] sec. Total target timesteps 161


IntProgress(value=0, description='Read paw data:', max=63)

IntProgress(value=0, description='Read lick data:', max=133)

Lick trial count inconsistent with neuronal 105 106
Inferred from behaviour [41, 36, 14, 14]
Inferred from lick      [41, 36, 14, 14, 0]
Lick trial count inconsistent with neuronal 162 163
Inferred from behaviour [68, 63, 23, 8]
Inferred from lick      [68, 63, 23, 8, 0]
Lick trial count inconsistent with neuronal 339 340
Inferred from behaviour [148, 138, 49, 4]
Inferred from lick      [148, 138, 49, 4, 0]
Lick trial count inconsistent with neuronal 46 47
Inferred from behaviour [20, 12, 11, 3]
Inferred from lick      [20, 12, 11, 3, 0]
Lick trial count inconsistent with neuronal 430 431
Inferred from behaviour [197, 216, 1, 16]
Inferred from lick      [197, 216, 1, 16, 0]
Lick trial count inconsistent with neuronal 438 437
Inferred from behaviour [116, 96, 140, 86]
Inferred from lick      [116, 96, 140, 86, 0]
Trial count inside file inconsistent 939 940 940 940
Lick trial count inconsistent with neuronal 940 939
Inferred from behaviour [248, 183, 458, 50]
Inferred from lick      [24

IntProgress(value=0, description='Read whisk data:', max=129)

# Analysis

## 1. Correlation and Synchrony

In [16]:
# Read LVM file from command line

synchrony_by_day = {}

for mouse in mice:
    synchrony_by_day[mouse] = {"date" : [], "sychr" : []}
    
    pathIdxs = np.where((micePathArr[:, 0] == mouse)&(micePathArr[:, 1] == 'data_path'))[0]
    
    for pathIdx in pathIdxs[:1]:
        # Get data
        dataIdx = findDataIdx(pathIdx)
        data = dataNeuronal[dataIdx]

        # Get parameters
        nTrials, nTimes, nChannels = data.shape

        data2D = []

        for iCh in range(nChannels):
            dataChannel = []
            for iTr in range(nTrials):
                dataChannel += [data[iTr, :, iCh] - np.mean(data[iTr, :, iCh])]
            data2D += [np.array(dataChannel).flatten()]

        # Compute normalization variances
        s_data = [np.sqrt(np.correlate(d, d)) for d in data2D]

        # Compute correlation
        cr = np.zeros((nChannels, nChannels))
        for iCh in range(nChannels):
            for jCh in range(iCh+1, nChannels):
                cr[iCh][jCh] = np.correlate(data2D[iCh], data2D[jCh]) / (s_data[iCh] * s_data[jCh])

        dateThis = np.array(micePathArr[pathIdx, 3].split("_")[2:5], dtype=int)
        synchrony_by_day[mouse]["date"] = dateThis
        synchrony_by_day[mouse]["sychr"] += [np.sum(np.abs(cr)) * 2 / nChannels / (nChannels - 1)]
    
print(synchrony_by_day)
    
# plt.figure()
# plt.plot(synchrony_by_day)
# plt.show()

{'mtp_11': {'date': array([2016,    9,    6]), 'sychr': [0.2796691850677035]}, 'mtp_16': {'date': array([2018,    5,   16]), 'sychr': [0.6098886603407517]}, 'mtp_12': {'date': array([2016,   10,   26]), 'sychr': [0.35247005685144245]}, 'mtp_10': {'date': array([2016,    9,    7]), 'sychr': [0.24570216069865228]}, 'mtp_9': {'date': [], 'sychr': []}, 'mtp_1': {'date': array([2015,    6,   18]), 'sychr': [0.5433900310624592]}, 'mtp_7': {'date': array([2016,    2,   18]), 'sychr': [0.38196770307663586]}, 'mtp_8': {'date': [], 'sychr': []}, 'mtp_2': {'date': array([2015,    6,   19]), 'sychr': [0.48207846985099834]}, 'mtp_13': {'date': array([2017,    3,    8]), 'sychr': [0.5106373876026021]}, 'mtp_15': {'date': array([2018,    5,   16]), 'sychr': [0.4452706283397996]}}
